In [3]:
# the original code is in the SentenceClassification.ipytn file. I am eliminating many variables in this file.
# This file is copied from the SentenceClassification_main file.
# In here we are trying to extract the data related to the value "Health" in the category Personal Information Type

import glob
import pandas as pd
import json
import csv
from bs4 import BeautifulSoup, NavigableString
import nltk.data

path = r'Plain_Html'  # use your path
allFiles = glob.glob(path + "/*.html")
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

path = r'Train'  # use your path
allFiles = glob.glob(path + "/*.csv")



allFiles = allFiles[:65]


# all the followint positives are to be counted if the sentences are in that particular class. 
# A sen_neg to be counted in any other circumstances
sen0_pos = 0 # action_first_party no. of sentence positives
sen0_neg = 0 # action_first_party no. of sentence negatives
sen_actioFP = []
sen_actioFP_class = []

sen1_pos = 0 # Personal Information Type no. of sentence positives
sen1_neg = 0 # action_first_party no. of sentence negatives
sen_personalIT = []
sen_personalIT_class = []

sen2_pos = 0 # purpose no. of sentence negatives
sen2_neg = 0 # purpose no. of sentence negatives
sen_purpose = []
sen_purpose_class = []

sen3_pos = 0

def html_to_text(html):
    "Creates a formatted text email message as a string from a rendered html template (page)"
    html_report_part1 = open(html,'r')
    soup = BeautifulSoup(html_report_part1, 'html.parser')
    # Ignore anything in head tag
    text = []
    start_index = []
    end_index = []
    for element in soup.descendants:
        # We use type and not isinstance since comments, cdata, etc are subclasses that we don't want
        count = 0;
        if type(element) == NavigableString:
            # We use the assumption that other tags can't be inside a script or style
            if element.parent.name in ('script', 'style'):
                continue

            # remove any multiple and leading/trailing whitespace
            string = ' '.join(element.string.split())
            if string:
                if element.parent.name == 'a':
                    a_tag = element.parent
                    # replace link text with the link
                    string = a_tag['href']
                    # concatenate with any non-empty immediately previous string
                    if (    type(a_tag.previous_sibling) == NavigableString and
                            a_tag.previous_sibling.string.strip() ):
                        text[-1] = text[-1] + ' ' + string
                        continue
                elif element.previous_sibling and element.previous_sibling.name == 'a':
                    text[-1] = text[-1] + ' ' + string
                    continue
                elif element.parent.name == 'p':
                    # Add extra paragraph formatting newline
                    string = '\n' + string
                text += [string]
                start_index.append(count)
    doc = '\n'.join(text)
    return doc



for file in allFiles:
    
    df = pd.read_csv(file, thousands=',', header=None)
    len(df)
    number_of_segments = len(df) + 1

    file = file.split('Train/', 1)[1]

    ### beautifulsuop on html doc
    file_html = 'Plain_Html/'+file
    file_html = file_html.split('csv', 1)[0]
    file_html = file_html+'html'
    ls = []
    indeces = []
    data = html_to_text(file_html)
    ls = tokenizer.tokenize(data)
    count = 0
    for l in ls:
        indeces.append([count,count+len(l)])
        count = count+len(l)

    for i in range(number_of_segments-1):
        choice = 0 #whether the sentence is from one of the following categories
        if df[5][i] == "First Party Collection/Use":
            parse_json = json.loads(str(df[6][i]))
            # parsing and fetching the text related to "Action First Party"            
#             if parse_json["Action First-Party"]["endIndexInSegment"] != -1:
#                 choice = 1
#                 str_index = parse_json["Action First-Party"]["startIndexInSegment"]
#                 end_index = parse_json["Action First-Party"]["endIndexInSegment"]
#                 for index in range(len(ls)):
#                     parsed_sen = ls[index].replace("/\n|||-","")                        
#                     parsed_sen = ls[index].replace("|||","") 
#                     if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
#                        (str_index < indeces[index][0] and end_index > indeces[index][1]) or
#                        (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
#                        (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):                        
#                         sen_actioFP.append(parsed_sen)
# #                         sen_personalIT.append(ls[index])
# #                         sen_purpose.append(ls[index])
# #                         sen_actioFP_class.append("pos")                        
# #                         sen_personalIT_class.append("neg")                        
# #                         sen_purpose_class.append("neg")
#                         sen0_pos += 1
#                         sen1_neg += 1
#                         sen2_neg += 1
# #                     else: 
# #                         sen_actioFP.append(parsed_sen)
# #                         sen_personalIT.append(ls[index])
# #                         sen_purpose.append(ls[index])
# #                         sen_actioFP_class.append("neg")                        
# #                         sen_personalIT_class.append("neg")                        
# #                         sen_purpose_class.append("neg")
# #                         sen0_neg += 1
# #                         sen1_neg += 1
# #                         sen2_neg += 1
            # parsing and fetching the text related to "Personal Information Type"                        
            if parse_json["Personal Information Type"]["endIndexInSegment"] != -1 and parse_json["Personal Information Type"]["value"]=="Health":
                choice = 1
                str_index = parse_json["Personal Information Type"]["startIndexInSegment"]
                end_index = parse_json["Personal Information Type"]["endIndexInSegment"]
                
                for index in range(len(ls)):
                    parsed_sen = ls[index].replace("/\n|||-","")                        
                    parsed_sen = ls[index].replace("|||","") 
                    if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
                       (str_index < indeces[index][0] and end_index > indeces[index][1]) or
                       (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
                       (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):                        
#                         sen_actioFP.append(parsed_sen)
                        sen_personalIT.append([parsed_sen,parse_json["Personal Information Type"]["value"]])
#                         sen_purpose.append(ls[index])
#                         sen_actioFP_class.append("neg")                        
#                         sen_personalIT_class.append("pos")                        
#                         sen_purpose_class.append("neg")  
                        sen0_neg += 1
                        sen1_pos += 1
                        sen2_neg += 1
#                     else: 
#                         sen_actioFP.append(parsed_sen)
#                         sen_personalIT.append(ls[index])
#                         sen_purpose.append(ls[index])
#                         sen_actioFP_class.append("neg")                        
#                         sen_personalIT_class.append("neg")                        
#                         sen_purpose_class.append("neg")  
#                         sen0_neg += 1
#                         sen1_neg += 1
#                         sen2_neg += 1
            # parsing and fetching the text related to "Purpose"                
#             if parse_json["Purpose"]["endIndexInSegment"] != -1:
#                 choice = 1            
#                 str_index = parse_json["Purpose"]["startIndexInSegment"]
#                 end_index = parse_json["Purpose"]["endIndexInSegment"]
#                 for index in range(len(ls)):
#                     parsed_sen = ls[index].replace("/\n|||-","")                        
#                     parsed_sen = ls[index].replace("|||","") 
#                     if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
#                        (str_index < indeces[index][0] and end_index > indeces[index][1]) or
#                        (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
#                        (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):
# #                         sen_actioFP.append(parsed_sen)
# #                         sen_personalIT.append(ls[index])
#                         sen_purpose.append(parsed_sen)
# #                         sen_actioFP_class.append("neg")                        
# #                         sen_personalIT_class.append("neg")                        
# #                         sen_purpose_class.append("pos")    
#                         sen0_neg += 1
#                         sen1_neg += 1
#                         sen2_pos += 1
# #                     else: 
# #                         sen_actioFP.append(parsed_sen)
# #                         sen_personalIT.append(ls[index])
# #                         sen_purpose.append(ls[index])
# #                         sen_actioFP_class.append("neg")                        
# #                         sen_personalIT_class.append("neg")                        
# #                         sen_purpose_class.append("neg")                        
# #                         sen0_neg += 1
# #                         sen1_neg += 1
# #                         sen2_neg += 1
                                
#         if df[5][i] == "Third Party Sharing/Collection":
#             parse_json = json.loads(str(df[6][i]))
#             if parse_json["Personal Information Type"]["endIndexInSegment"] != -1:
#                 choice = 1
#                 str_index = parse_json["Personal Information Type"]["startIndexInSegment"]
#                 end_index = parse_json["Personal Information Type"]["endIndexInSegment"]
#                 for index in range(len(ls)):                    
#                     if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
#                        (str_index < indeces[index][0] and end_index > indeces[index][1]) or
#                        (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
#                        (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):
#                         sen0_neg += 1
#                         sen1_pos += 1
#                         sen2_neg += 1
#                     else: 
#                         sen0_neg += 1
#                         sen1_neg += 1                        
#                         sen2_neg += 1

#             if parse_json["Purpose"]["endIndexInSegment"] != -1:
#                 choice = 1
#                 total_data_samples2.append(parse_json["Purpose"]["selectedText"])
#                 total_data_number2.append(2)
#                 total_data_label2.append("Purpose") 
#                 str_index = parse_json["Purpose"]["startIndexInSegment"]
#                 end_index = parse_json["Purpose"]["endIndexInSegment"]
#                 for index in range(len(ls)):
#                     if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
#                        (str_index < indeces[index][0] and end_index > indeces[index][1]) or
#                        (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
#                        (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):
#                         sen0_neg += 1
#                         sen1_neg += 1
#                         sen2_pos += 1
#                     else: 
#                         sen0_neg += 1
#                         sen1_neg += 1                        
#                         sen2_neg += 1

#         if df[5][i] == "User Choice/Control":
#             parse_json = json.loads(str(df[6][i]))
#             if parse_json["Personal Information Type"]["endIndexInSegment"] != -1:
#                 choice = 1
#                 str_index = parse_json["Personal Information Type"]["startIndexInSegment"]
#                 end_index = parse_json["Personal Information Type"]["endIndexInSegment"]
#                 for index in range(len(ls)):
#                     if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
#                        (str_index < indeces[index][0] and end_index > indeces[index][1]) or
#                        (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
#                        (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):
#                         sen0_neg += 1
#                         sen1_pos += 1
#                         sen2_neg += 1
#                     else: 
#                         sen0_neg += 1
#                         sen1_neg += 1                        
#                         sen2_neg += 1
            
#             if parse_json["Purpose"]["endIndexInSegment"] != -1:
#                 choice = 1
#                 str_index = parse_json["Purpose"]["startIndexInSegment"]
#                 end_index = parse_json["Purpose"]["endIndexInSegment"]
#                 for index in range(len(ls)):
#                     if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
#                        (str_index < indeces[index][0] and end_index > indeces[index][1]) or
#                        (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
#                        (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):
#                         sen0_neg += 1
#                         sen1_neg += 1
#                         sen2_pos += 1
#                     else: 
#                         sen0_neg += 1
#                         sen1_neg += 1                        
#                         sen2_neg += 1

#         if df[5][i] == "Data Retention":
#             parse_json = json.loads(str(df[6][i]))
#             if parse_json["Personal Information Type"]["endIndexInSegment"] != -1:
#                 choice = 1
#                 str_index = parse_json["Personal Information Type"]["startIndexInSegment"]
#                 end_index = parse_json["Personal Information Type"]["endIndexInSegment"]
#                 for index in range(len(ls)):
#                     if((str_index > indeces[index][0] and end_index < indeces[index][1]) or 
#                        (str_index < indeces[index][0] and end_index > indeces[index][1]) or
#                        (str_index < indeces[index][0] and end_index > indeces[index][0] and end_index < indeces[index][1]) or
#                        (str_index > indeces[index][0] and str_index < indeces[index][1] and end_index > indeces[index][1])):
#                         sen0_neg += 1
#                         sen1_pos += 1                        
#                         sen2_neg += 1
#                     else: 
#                         sen0_neg += 1
#                         sen1_neg += 1                        
#                         sen2_neg += 1
                            
                            
# print("----- postive and -ve sentences")                
# print(len(sen_actioFP),len(sen_actioFP_class))
# print(len(sen_personalIT),len(sen_personalIT_class))
# print(len(sen_purpose),len(sen_purpose_class))
# print(len(sentence_list))


# print(sen0_pos, sen0_neg)
# print(sen1_pos, sen1_neg)
# print(sen2_pos, sen2_neg)


print("len of the health sentences from Personal Information Type")
print(len(sen_personalIT))

len of the health sentences from Personal Information Type
75


In [33]:

print(sen_actioFP[:10])
print(len(sen_actioFP))

['Privacy Policy\nSci-News.com is committed to protecting and respecting your privacy.', 'To better inform you of our policy concerning user privacy, we have adopted the following terms.', 'Privacy Policy\nSci-News.com is committed to protecting and respecting your privacy.', 'To better inform you of our policy concerning user privacy, we have adopted the following terms.', 'Privacy Policy\nSci-News.com is committed to protecting and respecting your privacy.', 'Privacy Policy\nSci-News.com is committed to protecting and respecting your privacy.', 'Privacy Policy\nSci-News.com is committed to protecting and respecting your privacy.', 'Please note that these terms are subject to change, and any such changes will be included on this page.', 'Please note that these terms are subject to change, and any such changes will be included on this page.', 'Please note that these terms are subject to change, and any such changes will be included on this page.']
4464


In [17]:
print(sen_purpose[:10])

['Information that Sci-News.com May Collect Online\nSci-News.com may collect and process the following data about you:\n- information that you provide by filling in forms on our site, including names, e-mail and website addresses; we may also ask you for information for other purposes, for example when you report a problem with our site;\n- if you contact us, we may keep a record of that correspondence;\n- details of your visits to our site including, but not limited to, traffic data, location data, weblogs and other communication data.', 'Information that Sci-News.com May Collect Online\nSci-News.com may collect and process the following data about you:\n- information that you provide by filling in forms on our site, including names, e-mail and website addresses; we may also ask you for information for other purposes, for example when you report a problem with our site;\n- if you contact us, we may keep a record of that correspondence;\n- details of your visits to our site including, 

In [37]:
# Extracted texts for each and every attribute



import subprocess
import os
import json

# To prevent butter from over-browning in your pan, add a little bit of lemon juice.

# lines = []
# while True:
#     line = input()
#     if line:
#         lines.append(line)
#     else:
#         break
text = '\n'.join(sen_actioFP[:200])

text_file = open("data/aa.txt", "w")
text_file.write("%s" % text)
text_file.close()

text_personalIT = '\n'.join(sen_personalIT[:200])

text_file = open("data/bb.txt", "w")
text_file.write("%s" % text_personalIT)
text_file.close()

text_purpose = '\n'.join(sen_purpose[:200])

text_file = open("data/cc.txt", "w")
text_file.write("%s" % text_purpose)
text_file.close()


In [32]:
# Extracting the frames and frame elements for 'action first party'
subprocess.call(['/Users/aryan/Desktop/personal_projects/semafor3/semafor/bin/runMalt.sh',
                '/Users/aryan/Desktop/masters/MachineLearningModels/data/actioFP.txt',
               '/Users/aryan/Desktop/masters/MachineLearningModels/data/'])

os.system('cat /Users/aryan/Desktop/masters/MachineLearningModels/data/conll | nc localhost 8081 > /Users/aryan/Desktop/masters/MachineLearningModels/data/output.json')



0

In [30]:
import json

count = 0
with open('/Users/aryan/Desktop/personal_projects/semafor3/data/output.json') as f:
    for line in f:
        print(sen_actioFP[count])
        data = json.loads(line)
        # ss = [data[i]['frames'][0]['annotationSets'][0]['frameElements'][0]['name'] for i in range(len(data))]
        frames1 = []
        frame_elements1 = []
#         print(data.get('frames'))
        for i in range(len(data.get('frames'))):
            print("Frame is --> ",data.get('frames')[i].get('target').get('name'))
            
            for j in range(len(data.get('frames')[i].get('target').get('spans'))):
                print("Text is --> ",data.get('frames')[i].get('target').get('spans')[j].get('text'))
                
            for k in range(len(data.get('frames')[i].get('annotationSets'))):
                for l in range(len(data.get('frames')[i].get('annotationSets')[k].get('frameElements'))):
                    print("Frame Element Name -->",data.get('frames')[i].get('annotationSets')[k].get('frameElements')[l].get('name'))
                    for m in range(len(data.get('frames')[i].get('annotationSets')[k].get('frameElements')[l].get('spans'))):
                        print("Frame Element Text -->",data.get('frames')[i].get('annotationSets')[k].get('frameElements')[l].get('spans')[m].get('text'))
        
        
    
    
            print("-------")
        count+=1
        print("-----------")
        print("-----------")
#             frames1.append(data['frames'][j]['target']['name'])
#             for k in range(len(data[i]['frames'][j]['annotationSets'][0]['frameElements'])):
#                 frame_elements1.append(data[i]['frames'][j]['annotationSets'][0]['frameElements'][k]['name'])
#         print(frames1)
#         print(frame_elements1)
#     print(data[i]['frames'])

# print(set(frames))



Privacy Policy
Sci-News.com is committed to protecting and respecting your privacy.
Frame is -->  Stimulus_focus
Text is -->  pleasant
Frame Element Name --> Stimulus
Frame Element Text --> morning
-------
Frame is -->  Calendric_unit
Text is -->  morning
-------
-----------
-----------
